In [1]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd

In [107]:
#parcel_number = '871288650'
parcel_number = '888058983'
query = f"""
SELECT opp.parcel_number, year_built, year_built_estimate, 
mailing_street, mailing_address_1,
building_code_description, category_code_description, 
n_violations, n_violations_open, n_violations_dangerous,
has_active_rental_license, expiration_date,
r1.recording_date, r1.grantees, r1.grantors, r1.cash_consideration, r1.property_count,
a1.market_value as market_value_latest_assessment, a1.latest_assessment_year
FROM opa_properties_public opp
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    
) v1
ON v1.parcel_number = opp.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations_open
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    AND violationstatus = 'OPEN'
    
) v2
ON v1.parcel_number = v2.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations_dangerous
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    AND caseprioritydesc in ('HAZARDOUS', 'UNSAFE', 'IMMINENTLY DANGEROUS')
    
) v3
ON v1.parcel_number = v3.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, 
    CASE WHEN count(*) > 0 THEN TRUE ELSE FALSE END as has_active_rental_license, 
    max(expirationdate) as expiration_date
    FROM business_licenses 
    WHERE opa_account_num = '{parcel_number}'
    AND licensetype='Rental'
    AND licensestatus='Active'
) l1
ON v1.parcel_number = l1.parcel_number
LEFT JOIN (
    SELECT opa_account_num as parcel_number, 
    *
    FROM rtt_summary 
    WHERE opa_account_num = '{parcel_number}'
    AND (
        document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK'
    )
    ORDER BY recording_date LIMIT 1
) r1
ON v1.parcel_number = r1.parcel_number
LEFT JOIN (
    SELECT parcel_number, 
    year as latest_assessment_year,
    market_value
    FROM assessments a
    WHERE parcel_number = '{parcel_number}'
    ORDER BY year DESC LIMIT 1
) a1
ON v1.parcel_number = a1.parcel_number
WHERE opp.parcel_number = '{parcel_number}'

"""

df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)

In [139]:
#parcel_number = '871288650'
parcel_number = '888058983'
query = f"""
    SELECT opa_account_num as parcel_number, 
    grantees, grantors, cash_consideration, recording_date, property_count, condo_name
    FROM rtt_summary 
    WHERE opa_account_num = '{parcel_number}'
    AND (
        document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK'
    )
    ORDER BY recording_date DESC

"""

df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'parcel_number': '888058983',
  'grantees': '14 WA PARTNERS LP',
  'grantors': '1401 WALNUT STREET UNIT B LLC',
  'cash_consideration': 14000000,
  'recording_date': '2013-06-27T00:00:00Z',
  'property_count': 7,
  'condo_name': '1401 WALNUT TOWER'},
 {'parcel_number': '888058983',
  'grantees': '1401 WALNUT STREET UNIT B LLC',
  'grantors': 'K HOVNANIAN AT BROAD AND WALNUT LLC',
  'cash_consideration': 20000000,
  'recording_date': '2007-08-22T00:00:00Z',
  'property_count': 8,
  'condo_name': 'WALNUT TOWERS'},
 {'parcel_number': '888058983',
  'grantees': 'K HOVNANIAN AT BROAD AND WALNUT LLC',
  'grantors': '1401 WALNUT PARTNERS L P',
  'cash_consideration': 1450000,
  'recording_date': '2006-06-29T00:00:00Z',
  'property_count': 2,
  'condo_name': 'WALNUT TOWERS'}]